In [1]:
import torch
import pytorch_lightning as pl
pl.seed_everything(100)
import os
import numpy as np
import wandb
from io import BytesIO
from PIL import Image
from pytorch_lightning.loggers import WandbLogger
import torch.nn.functional as F
from torchvision import transforms
import torchmetrics
from torch.utils.data import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
from torchvision import models
import cv2

C:\Users\rsorokin\Anaconda3\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\rsorokin\Anaconda3\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")
Global seed set to 100


In [2]:
wandb.login()

wandb: Currently logged in as: chrommium. Use `wandb login --relogin` to force relogin


True

In [3]:
TRAIN_IMGS_PATH = '../prepare_datasets/data_add_all_balanced/train/'
TRAIN_ADD_IMGS_PATH = '../prepare_datasets/data_add_all_balanced/train_add/'
TRAIN_ALL_IMGS_PATH = '../prepare_datasets/data_add_all_balanced/train_all_add/'


TRAIN_DF_PATH = '../prepare_datasets/data_add_all_balanced/train_clear.csv'
TRAIN_ADD_DF_PATH = '../prepare_datasets/data_add_all_balanced/train_add.csv'
TRAIN_ALL_DF_PATH = '../prepare_datasets/data_add_all_balanced/train_all_add.csv'

In [4]:
train_transform = transforms.Compose([
    transforms.Resize((384,384)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                          std=[0.229, 0.224, 0.225]),
])

valid_transform = transforms.Compose([
    transforms.Resize((384,384)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                          std=[0.229, 0.224, 0.225]),
])

In [5]:
class ImageDatasetTrain(Dataset):
    def __init__(self, data_df, transform=None):

        self.data_df = data_df
        self.transform = transform

    def __getitem__(self, idx):
        # достаем имя изображения и ее лейбл
        image_name, label = self.data_df.iloc[idx]['ID_img'], self.data_df.iloc[idx]['class']

        # читаем картинку. read the image
        image = cv2.imread(f"{TRAIN_ALL_IMGS_PATH}/{image_name}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(image)
        
        # преобразуем, если нужно. transform it, if necessary
        if self.transform:
            image = self.transform(image)
        
        return image, torch.tensor(label).long()
    
    def __len__(self):
        return len(self.data_df)

In [6]:
class ImageDatasetVal(Dataset):
    def __init__(self, data_df, transform=None):

        self.data_df = data_df
        self.transform = transform

    def __getitem__(self, idx):
        # достаем имя изображения и ее лейбл
        image_name, label = self.data_df.iloc[idx]['ID_img'], self.data_df.iloc[idx]['class']

        # читаем картинку. read the image
        image = cv2.imread(f"{TRAIN_ALL_IMGS_PATH}/{image_name}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(image)
        
        # преобразуем, если нужно. transform it, if necessary
        if self.transform:
            image = self.transform(image)
        
        return image, torch.tensor(label).long()
    
    def __len__(self):
        return len(self.data_df)

In [7]:
class ResNetFinetuner(pl.LightningModule):
    def __init__(self, resnet_version, train_dataset, val_dataset, batch_size, optimizer, optimizer_params, transfer=True):
        super(ResNetFinetuner, self).__init__()
        
        resnets = {
            18: models.resnet18, 34: models.resnet34,
            50: models.resnet50, 101: models.resnet101,
        }
        optimizers = {'adam': torch.optim.Adam}

        self.resnet_model = resnets[resnet_version](pretrained=True)
        self.train_dataset = train_dataset
        self.val_dataset = val_dataset
        self.optimizer_params = optimizer_params
        self.optimizer = optimizers[optimizer]
        self.batch_size = batch_size
        linear_size = list(self.resnet_model.children())[-1].in_features
        self.resnet_model.fc = torch.nn.Linear(linear_size, 3)
        
        if transfer:
            for child in list(self.resnet_model.children())[:-1]:
                for param in child.parameters():
                    param.requires_grad = False

        self.save_hyperparameters()
        
        self.train_f1 = torchmetrics.F1Score(3)
        self.val_f1 = torchmetrics.F1Score(3)
        self.train_acc = torchmetrics.Accuracy(3)
        self.val_acc = torchmetrics.Accuracy(3)
        
    def forward(self, X):
        X = self.resnet_model(X)
        return F.softmax(X, dim=1)

    def training_step(self, batch, batch_idx):
        x, y = batch
        preds = self(x)
        loss = F.cross_entropy(preds, y)
        self.log('train/loss', loss, on_epoch=True)
        self.train_f1(preds, y)
        self.train_acc(preds, y)
        self.log('train/f1', self.train_f1, on_epoch=True)
        self.log('train/acc', self.train_acc, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        preds = torch.argmax(logits, 1)
        self.val_f1(preds, y)
        self.val_acc(preds, y)
        self.log("validation/loss_epoch", loss)
        self.log("validation/f1_epoch", self.val_f1)
        self.log("validation/acc_epoch", self.val_acc)
        return logits

    def validation_epoch_end(self, validation_step_outputs):
        flattened_logits = torch.flatten(torch.cat(validation_step_outputs))
        self.logger.experiment.log(
            {"validation/logits": wandb.Histogram(flattened_logits.cpu()),
             "global_step": self.global_step}, commit=False
        )

#     def test_step(self, batch, batch_idx):
#         x, y, _ = batch
#         preds = self(x)
#         loss = F.cross_entropy(preds, y)
#         self.test_f1(preds, y)
#         self.log("test/loss_epoch", loss, on_step=False, on_epoch=True)
#         self.log("test/f1_epoch", self.test_f1, on_step=False, on_epoch=True)


    def configure_optimizers(self):
        return self.optimizer([p for p in self.parameters() if p.requires_grad], **self.optimizer_params)

    def train_dataloader(self):
        train_loader = torch.utils.data.DataLoader(self.train_dataset, 
                                                   batch_size=self.batch_size, 
                                                  shuffle=True)
        return train_loader

    def val_dataloader(self):
        val_loader = torch.utils.data.DataLoader(self.val_dataset, batch_size=self.batch_size)
        return val_loader
    
#     def test_dataloader(self):
#         train_loader = torch.utils.data.DataLoader(self.test_dataset, batch_size=self.batch_size)
#         return train_loader

In [8]:
class ImageLogger(pl.Callback):
    def __init__(self, val_samples, num_samples=32):
        super().__init__()
        self.val_imgs, self.val_labels = val_samples
        self.val_imgs = self.val_imgs[:num_samples]
        self.val_labels = self.val_labels[:num_samples]

    def on_validation_epoch_end(self, trainer, pl_module):
        val_imgs = self.val_imgs.to(device=pl_module.device)
        logits = pl_module(val_imgs)
        preds = torch.argmax(logits, 1)
        trainer.logger.experiment.log({
            "examples": [wandb.Image(x, caption=f"Pred: {pred}, Label: {y}") for x, pred, y in zip(val_imgs, preds, self.val_labels)],
            "global_step": trainer.global_step
        }, commit=False)


In [9]:
data_df = pd.read_csv(TRAIN_DF_PATH)
data_df['class'].value_counts()

1.0    308
2.0    143
0.0     92
Name: class, dtype: int64

# train full data

In [26]:
_, valid_df = train_test_split(data_df, test_size=0.2, random_state=43, stratify=data_df['class'])
train_df = pd.read_csv(TRAIN_ALL_DF_PATH)

train_dataset = ImageDatasetTrain(train_df, train_transform)
valid_dataset = ImageDatasetVal(valid_df, valid_transform)

In [27]:
wandb_logger = WandbLogger(project="lit-wandb")
optimizer_params = {'lr': 0.001}
resnet_finetuner = ResNetFinetuner(resnet_version=50, 
                                   train_dataset=train_dataset, 
                                   val_dataset=valid_dataset, 
                                   batch_size=32, 
                                   optimizer='adam', 
                                   optimizer_params=optimizer_params, 
                                   transfer=True)



samples = next(iter(resnet_finetuner.val_dataloader()))
trainer = pl.Trainer(
    logger=wandb_logger,    # W&B integration
    log_every_n_steps=50,   # set the logging frequency
    gpus=-1,                # use all GPUs
    max_epochs=3,           # number of epochs
    deterministic=True,     # keep it deterministic
    callbacks=[ImageLogger(samples)] # see Callbacks section
    )   
trainer.fit(resnet_finetuner)
#trainer.test(test_dataloaders=resnet_finetuner.test_dataloader())
wandb.finish()

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



  | Name         | Type     | Params
------------------------------------------
0 | resnet_model | ResNet   | 23.5 M
1 | train_f1     | F1Score  | 0     
2 | val_f1       | F1Score  | 0     
3 | train_acc    | Accuracy | 0     
4 | val_acc      | Accuracy | 0     
------------------------------------------
6.1 K     Trainable params
23.5 M    Non-trainable params
23.5 M    Total params
94.057    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

C:\Users\rsorokin\Anaconda3\lib\site-packages\pytorch_lightning\trainer\data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
Global seed set to 100
C:\Users\rsorokin\Anaconda3\lib\site-packages\pytorch_lightning\trainer\data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

epoch,▁▁▁▃▃▃▆▆▆███
global_step,▁▃▄▆█
train/acc_epoch,▁▆▇█
train/acc_step,▄▂▁█
train/f1_epoch,▁▆▇█
train/f1_step,▄▂▁█
train/loss_epoch,█▃▁▁
train/loss_step,█▇▇▁
trainer/global_step,▁▁▁▃▃▃▅▆▆▇██
validation/acc_epoch,▁▄█▄
validation/f1_epoch,▁▄█▄


# predictions

In [28]:
test_df = pd.read_csv("../sample_solution.csv")
test_df = test_df.drop(["class"], axis = 1)
test_df

,ID_img
0,34020749806_42065966214_42113475048_2
1,80128313599_98196458454_79029076007_8
2,17820331238_48919943775_53688855463_7
3,70492442702_21083599816_22777758696_0
4,94790217016_17108156014_60668676818_2
...,...
220,60879177998_15763718934_82574532042_2
221,11758169966_65799840524_72283028069_1
222,9259096884_2251720133_44072689872_8
223,37732252922_9265441355_19052721018_3


In [29]:
TEST_IMGS_PATH = '../test_dataset_test/'

class TestImageDataset(Dataset):
    def __init__(self, data_df, transform=None):
        self.data_df = data_df
        self.transform = transform

    def __getitem__(self, idx):
        image_name = self.data_df.iloc[idx]['ID_img']
        
        # читаем картинку
        image = cv2.imread(f"{TEST_IMGS_PATH}/{image_name}.jpg")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(image)
#         visualize(image)
        
        # преобразуем, если нужно
        if self.transform:
#             image = self.transform(image=image)['image']
            image = self.transform(image)

        return image
    
    def __len__(self):
        return len(self.data_df)

In [30]:
test_dataset = TestImageDataset(test_df, valid_transform)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=1,
                                           # shuffle=True,
                                           pin_memory=True)

In [31]:
from tqdm import tqdm

resnet_finetuner.eval()
predicts = []

for imgs in tqdm(test_loader):
    
    imgs = imgs
    pred = resnet_finetuner(imgs)

    for class_obj in pred:
        index, max_value = max(enumerate(class_obj), key=lambda i_v: i_v[1])
        predicts.append(index)
#         print(index)
#         plt.show()

100%|████████████████████████████████████████████████████████████████████████████████| 225/225 [01:53<00:00,  1.97it/s]


In [32]:
test_df['class'] = predicts

In [33]:
test_df.to_csv('./submit_resnet_balanced_6.csv', index=False) 

In [34]:
test_df

,ID_img,class
0,34020749806_42065966214_42113475048_2,2
1,80128313599_98196458454_79029076007_8,1
2,17820331238_48919943775_53688855463_7,2
3,70492442702_21083599816_22777758696_0,2
4,94790217016_17108156014_60668676818_2,2
...,...,...
220,60879177998_15763718934_82574532042_2,2
221,11758169966_65799840524_72283028069_1,1
222,9259096884_2251720133_44072689872_8,0
223,37732252922_9265441355_19052721018_3,1


In [35]:
test_df['class'].value_counts(normalize=True)

1    0.573333
2    0.235556
0    0.191111
Name: class, dtype: float64